In [ ]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.0/603.0 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

import pathlib
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.llms import HuggingFacePipeline
import torch

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from google.colab import userdata
import os
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = userdata.get("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "HUGGINGFACEHUB_API_TOKEN"

In [ ]:
loader = PyPDFDirectoryLoader("/content/sample_data/data")
doc = loader.load()

In [ ]:
import re

def clean_text(doc):
    text = doc.page_content
    # فصل الأرقام عن الحروف (رقم يليه حرف أو حرف يليه رقم)
    text = re.sub(r'(\d+)([a-zA-Z])', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z])(\d+)', r'\1 \2', text)

     # إزالة الحروف الخاصة مثل "-" فقط داخل الكلمات
    text = re.sub(r'(\w)-(\w)', r'\1\2', text)

    # إزالة الكلمات المتكررة بجانب بعضها
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

    # إضافة مسافة بين الأحرف إذا كان الحرف الثاني كبيرًا
    #text = re.sub(r'(\w)([A-Z])', r'\1 \2', text)

    # Remove extra whitespace
    #text = re.sub(r'\s+', ' ', text).strip()

    # Remove any links
    text = re.sub(r'http\S+', ' ', text)

    # Remove any email addresses
    text = re.sub(r'\S+@\S+', ' ', text)

    # Remove any special characters except for letters and digits
    text = re.sub(r'[^A-Za-z0-9 ]+', ' ', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove any multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

docs =list(map(lambda x: clean_text(x), doc))
docs

['',
 'OXFORD MEDICAL PUBLICATIONS Oxford Handbook of Clinical Pharmacy 00Wiffen FM indd i 11 10 2011 9 44 43 AM',
 'Oxford Handbook for the Foundation Programme 3 e Oxford Handbook of Acute Medicine 3 e Oxford Handbook of Anaesthesia 3 e Oxford Handbook of Applied Dental Sciences Oxford Handbook of Cardiology 2 e Oxford Handbook of Clinical and Laboratory Investigation 3 e Oxford Handbook of Clinical Dentistry 5 e Oxford Handbook of Clinical Diagnosis 2 e Oxford Handbook of Clinical Examination and Practical Skills Oxford Handbook of Clinical Haematology 3 e Oxford Handbook of Clinical Immunology and Allergy 2 e Oxford Handbook of Clinical Medicine Mini Edition 8 e Oxford Handbook of Clinical Medicine 8 e Oxford Handbook of Clinical Pharmacy 2 e Oxford Handbook of Clinical Rehabilitation 2 e Oxford Handbook of Clinical Specialties 8 e Oxford Handbook of Clinical Surgery 3 e Oxford Handbook of Complementary Medicine Oxford Handbook of Critical Care 3 e Oxford Handbook of Dental Patient

In [ ]:
import os
from PyPDF2 import PdfReader
import re

def extract_text_from_pdf(file_path):

    reader = PdfReader(file_path)
    all_text = []
    for page_num, page in enumerate(reader.pages, start=1):
        text = page.extract_text()
        if text:
            cleaned_text = clean_text(text)
            all_text.append((page_num, cleaned_text))
    return all_text

def clean_text(text):
    # إزالة المسافات الزائدة والأسطر الفارغة
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    # Remove any links
    text = re.sub(r'http\S+', ' ', text)
    # Remove any email addresses
    text = re.sub(r'\S+@\S+', ' ', text)
    # استبدال الأحرف غير المدعومة
    text = text.replace('—', '-').replace('“', '"').replace('”', '"').replace('’', "'")
    return text

def get_text_from_pdfs(directory_path):

    all_text_data = {}
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.pdf'):
            file_path = os.path.join(directory_path, file_name)

            text_data = extract_text_from_pdf(file_path)
            all_text_data[file_name] = text_data
    return all_text_data


directory_path = '/content'


docs = get_text_from_pdfs(directory_path)


In [ ]:
docs

{'Oxford_American_Handbook_of_Clinical_Dia_removed.pdf': [(1,
   'Chapter 1 1 The diagnostic process Introduction 2 Developing a differential diagnosis 3 Problem-solving skills 4 Dynamic diagnoses and the possibility of cognitive error 12 Summary 14'),
  (2,
   'CHAPTER 1 The diagnostic process 2 Introduction On the eve of clinical clerkships, at the end of basic science training, the medical student has mastered more information than he or she will ever again know in a lifetime. Yet it is precisely at this time that developing the clinical scenario is most awkward. Simply put, the problem is one of incompatibility of two sets of knowledge. Medical knowledge comprises millions of facts. It must be organized by disease state and by pathophysiology. Information is easily retrieved, once mastered, if organized by disease state or pathogenesis. Clinical information, obtained from a single patient during the interview and exam, similarly comprises hundreds, if not thousands, of facts. The p

In [ ]:
from langchain.schema import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
# Convert each string in cleaned_docs to a Document object
documents = [Document(page_content=doc) for doc in docs]
chunks = text_splitter.split_documents(documents)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")


<ipython-input-8-e470369191f5>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct" , device_map="auto")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.3,
    top_p=1
)

In [ ]:
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Define the prompt template
template = """
<|context|>
You are an AI assistant that your name is 'MM3' follows instructions extremely well.
Please be truthful and give direct answers.
<|user|>
{query}
</s>
<|assistant|>
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever, "query": lambda x: x}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
!pip install gradio


In [ ]:
!pip install gtts

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
from gtts import gTTS
import os

In [ ]:
# Initialize the ASR model for speech-to-text
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en",device="cuda")

# Function to transcribe audio input
def transcribe(audio):
    sr, y = audio

    # Convert to mono if stereo
    if y.ndim > 1:
        y = y.mean(axis=1)

    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    return transcriber({"sampling_rate": sr, "raw": y})["text"]


In [ ]:
# Function to convert text to audio using gTTS
def text_to_audio(text):
    tts = gTTS(text=text, lang='en')
    audio_file = "output.mp3"
    tts.save(audio_file)
    return audio_file


In [ ]:
def generate_answer(query):
    result = rag_chain.invoke(query)
    clean_result = result.replace("<|context|>", "").replace("<|user|>", "").replace("<|assistant|>", "").replace("</s>", "").strip()
    clean_result = clean_result.replace("Human:", "").replace("You are an AI assistant that your name is 'MM3' follows instructions extremely well.", "").replace("Please be truthful and give direct answers.", "").strip()
    clean_result = clean_result.replace(query, "").strip()
    return clean_result


In [ ]:
# Function to process input based on user selection
def process_input(selection, audio=None, text_input=None):
    if selection == "Voice" and audio is not None:
        transcribed_text = transcribe(audio)
    elif selection == "Text" and text_input:
        transcribed_text = text_input
    else:
        return "Please provide input based on your selected option.", None

    generated_answer = generate_answer(transcribed_text)

    audio_file = text_to_audio(generated_answer)

    return generated_answer, audio_file


In [ ]:
# Create the Gradio interface
with gr.Blocks() as demo:

    input_choice = gr.Radio(["Voice", "Text"], label="Choose Input Method", value="Voice")

    audio_input = gr.Audio(sources="microphone", type="numpy", label="Upload Audio", visible=True)
    text_input = gr.Textbox(placeholder="Enter text here...", label="Text Input", visible=False)

    def update_input(choice):
        if choice == "Voice":
            return gr.update(visible=True), gr.update(visible=False)
        elif choice == "Text":
            return gr.update(visible=False), gr.update(visible=True)

    input_choice.change(fn=update_input, inputs=input_choice, outputs=[audio_input, text_input])

    submit_btn = gr.Button("Submit")

    # Outputs for the answer
    output_text = gr.Textbox(label="Generated Text Answer")
    output_audio = gr.Audio(label="Generated Audio Answer")

    # Process input and generate the answer
    submit_btn.click(
        fn=process_input,
        inputs=[input_choice, audio_input, text_input],
        outputs=[output_text, output_audio]
    )

# Launch the Gradio interface
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://50d9af2c8317d59896.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
